# Application part (You need a trained model loaded)

In [ ]:
import numpy as np
import cv2
from IPython.display import clear_output

def choice (score):
    score_tmp = 0
    ind = 0
    for i in range(len(score)):
        if score[i] > score_tmp :
            score_tmp = score[i]
            ind = i
    return ind,score_tmp

def second_choice (score,max_tmp):
    score_tmp = 0
    for i in range(len(score)):
        if (score[i] > score_tmp) and (score[i] != max_tmp):
            score_tmp = score[i]
            ind = i
    return ind


name={0:"thumb_up",1:"thumb_down",2:"fist_1",
      3:"fist_2",4:"fist_3",5:"peace",6:"rock",7:"nice",
      8:"finger_up" ,9:"finger_up_2",
      10:"palm",11:"spok",12:"call"}

image={}
images_path = "variables/emojis/"#Path where the images of the emojis are stored (.png)
for n in name.values() : 
    image[n]= cv2.imread(images_path+n+".png")

cap = cv2.VideoCapture(0)
save = []
cap.set(3,200)
cap.set(4,200)
max_tmp = 0

while(True):
    # Capture frame-by-frame

    ret, frame = cap.read()
    # Our operations on the frame come here

    cv2.rectangle(frame,(0,0),(141,111),color=3)
    cv2.imshow('frame',frame)
    
    x_tmp = frame[:110,:140,:]

    if cv2.waitKey(1) == ord('q'):
        break
        
    x_tmp = x_tmp.reshape(1, 110, 140,3)
    pred = model_final.predict(x_tmp)
    name_first = name[choix(pred[0])[0]]
    max_tmp = choice(pred[0])[1]
    name_second = name[second_choice(pred[0],max_tmp)]

    cv2.imshow('First choice',image[name_first])
    cv2.imshow('Second choice',image[name_second])

cap.release()
cv2.destroyAllWindows()

# This part is used to capture the different images needed for the dataset
Press "c" to capture one image and press "q" to quit the application

In [ ]:
import numpy as np
import cv2
from IPython.display import clear_output

cap = cv2.VideoCapture(0)
save = []
cap.set(3,200)
cap.set(4,200)
max_tmp = 0

while(True):
    # Capture frame-by-frame

    ret, frame = cap.read()
    cv2.rectangle(frame,(0,0),(141,111),color=3)
    cv2.imshow('frame',frame)
    
    x_tmp = frame[:110,:140,:]

    if cv2.waitKey(1) == ord('c'):
        save.append(x_tmp)
        print(len(save))
        
    if cv2.waitKey(1) == ord('q'):
        break
    #i+=1
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [ ]:
name = "call" # The name you want to give to your captured emoji
path = "datasets/emojis/ # The path where you want to save the dataset
i =100
for e in save : 
    cv2.imwrite( +name+"/"+ str(i)+".jpg", e );
    i+=1    

# Data augmentation

In [ ]:
import Augmentor

# An array containing all the names you've given to your emojis
name=["thumb_up","thumb_down","fist_1","fist_2","fist_3","crossed_fingers","peace","rock","nice","finger_up","finger_up_2","palm","spok","call"]
dataset_path = 'datasets/emojis_2/'
for name in name :
    path = dataset_path +name   
    p = Augmentor.Pipeline(source_directory=path,output_directory="")
    p.random_distortion(probability=1, grid_width=4, grid_height=4, magnitude=8)
    p.flip_left_right(probability=0.5)
    p.sample(100)

# Opening of the data and splitting in test and valid

In [ ]:
import glob   
import cv2
import numpy as np

x = []
y = []
i = 0
img_width = 110
img_height = 140
name=["thumb_up","thumb_down","fist_1","fist_2","fist_3","peace","rock","nice","finger_up" ,"finger_up_2","palm","spok","call"]

for name in name :
    path = 'datasets/emojis_2/'+name+'/*.jpg'   
    files=glob.glob(path)   
    for file in files:  
        x.append(cv2.imread(file))
        y.append(i)
    i+=1     
    
from keras.utils import to_categorical
y = to_categorical(y)    
x = np.array(x)    
x = x.reshape(x.shape[0], img_width, img_height,3)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=42)


# Creation of the model and learning phase

In [ ]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 110, 140
batch_size = 50
epochs = 7

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:5]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(13, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

#,callbacks=[LearningRateScheduler(schedule, verbose=0)]
hist_transfer = model_final.fit(x_train, y_train, epochs=epochs,batch_size =batch_size,validation_split=0.1)


# Continue training of the model without having to create again

In [ ]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

hist_transfer = model_final.fit(x_train, y_train, epochs=3,batch_size =50,validation_split=0.1)


# Evaluation of the model

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = model_final.predict(x_test)
y_pred = y_pred.argmax(axis=-1)
y_true = np.argmax( y_test ,axis=1)

print(confusion_matrix(y_true,y_pred))
print(model_final.evaluate(x_test,y_test))

# Storing and opening of the model on the disk 

In [ ]:
def save_model(model,name,path):
    model_json = model.to_json()
    with open(path+name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
        model.save_weights(path+name+".h5")
        print("Saved model to disk")
        
def load_model(name,path):
    from keras.models import model_from_json
    # load json and create model
    json_file = open(path+name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(path+name+".h5")
    print("Loaded model from disk")
    return loaded_model


In [ ]:
save_model(model_final,"Transfer_final","models/emojis/")

In [ ]:
from keras import optimizers

model_final = load_model("Transfer_final","models/emojis/")
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])